# O intuito desse notebook e recuperar todas as imagens disponiveis  sobre pneumonia e covid e disponibilizar em um banco de dados


## bibliotecas necessárias

In [1]:
from imutils import paths
import pandas as pd
import shutil
import os

## variaveis iniciais

## Variaveis aonde está localizado os arquivos originais  de imagens baixados

In [2]:
def getTargetDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [9]:
dir_target_diretorio_raiz = getTargetDiretorio("MaquinaLocal")
dir_target_diretorio_dataset =r"{0}/{1}".format(dir_target_diretorio_raiz,'dataset')
dir_target_diretorio_imagem_covid =r"{0}/{1}".format(dir_target_diretorio_dataset,'covid')
dir_target_diretorio_pneumonia_normal =r"{0}/{1}".format(dir_target_diretorio_dataset,'normal')
dir_target_diretorio_pneumonia_pneumonia =r"{0}/{1}".format(dir_target_diretorio_dataset,'pneumonia')

dir_source_imagens_Covid =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/images'
dir_source_pneumonia =r'D:/Repos/pneumonia-chest_xray'

dir_source_pneumonia_normal = "{0}/{1}/{2}".format(dir_source_pneumonia,"test","NORMAL")  # 234 imagens
dir_source_pneumonia_pneumonia = "{0}/{1}/{2}".format(dir_source_pneumonia,"test","PNEUMONIA") # 390 imagens

dir_source_validacao_pneumonia_normal = "{0}/{1}/{2}".format(dir_source_pneumonia,"val","NORMAL")  # 234 imagens
dir_source_validacao_pneumonia_validacao = "{0}/{1}/{2}".format(dir_source_pneumonia,"val","PNEUMONIA") # 390 imagens


print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)
print("source")
print(dir_source_imagens_Covid)
print(dir_source_pneumonia_normal)
print(dir_source_pneumonia_pneumonia)
print(dir_source_validacao_pneumonia_normal)
print(dir_source_validacao_pneumonia_validacao)


target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia
source
D:/Repos/datasetsGitHub/covid-chestxray-dataset/images
D:/Repos/pneumonia-chest_xray/test/NORMAL
D:/Repos/pneumonia-chest_xray/test/PNEUMONIA
D:/Repos/pneumonia-chest_xray/val/NORMAL
D:/Repos/pneumonia-chest_xray/val/PNEUMONIA


## cria os diretorios se não existirem

In [6]:
lista_diretorios_target=[]
lista_diretorios_target.append(dir_target_diretorio_raiz)
lista_diretorios_target.append(dir_target_diretorio_dataset),
lista_diretorios_target.append(dir_target_diretorio_imagem_covid),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_normal),
lista_diretorios_target.append(dir_target_diretorio_pneumonia_pneumonia)

for item in lista_diretorios_target:
    if not os.path.exists(item):
        os.mkdir(item)
    




## variaveis do diretorio dos conjuntos de dados originais



## valida se o diretorio do git recebeu novas imagens


In [18]:
source_dir_metadata_local = r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/metadata.csv'

metadata_web_git ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'



In [19]:
df_git_covid_web  = pd.read_csv(metadata_web_git,error_bad_lines=False)

df_git_covid_local = pd.read_csv(source_dir_metadata_local,error_bad_lines=False)


print(len(df_git_covid_web))
print(len(df_git_covid_local))

if(len(df_git_covid_web) > len(df_git_covid_local)):
    print('temos {0} imagens com covid a mais no git web'.format(len(df_git_covid_web) - len(df_git_covid_local)))
elif(len(df_git_covid_web) < len(df_git_covid_local)):
    print('foram removidas {0} imagens  no git web'.format(len(df_git_covid_local) - len(df_git_covid_web)))


556
565
foram removidas 9 imagens  no git web


In [20]:
df_covid  = pd.read_csv(source_dir_metadata_local)

In [18]:
df_covid.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes
0,2,0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN


## populando 

## Importando nossas imagens para o banco de dados SqlLite

In [21]:
def clear_diretorio(diretorio):
    for file in os.listdir(diretorio):
        full_name =os.path.join(diretorio, file)
        
        if os.path.exists(full_name):
            os.remove(full_name) 
        
        

def get_imagens_covid(dataframe,diretorio):
        lista =[]
        for (i,row) in dataframe.iterrows():
                imagemName=row["filename"]
                image_source  = "{0}/{1}".format(diretorio,imagemName)
                lista.append(image_source)

        return lista


def get_imagens_diretorio(diretorio):
        imagePaths =[]
        imagePaths = list(paths.list_images(diretorio))
        imagePaths = [ item.replace("\\","/") for item in imagePaths]
        
        return imagePaths        


## Carregar as listas de imagens

In [22]:


print("source")
print(dir_source_imagens_Covid)
print(dir_source_pneumonia_normal)
print(dir_source_pneumonia_pneumonia)
print(dir_source_validacao_pneumonia_normal)
print(dir_source_validacao_pneumonia_validacao)
print("target")
print(dir_target_diretorio_raiz)
print(dir_target_diretorio_dataset)
print(dir_target_diretorio_imagem_covid)
print(dir_target_diretorio_pneumonia_normal)
print(dir_target_diretorio_pneumonia_pneumonia)



source
D:/Repos/datasetsGitHub/covid-chestxray-dataset/images
D:/Repos/pneumonia-chest_xray/test/NORMAL
D:/Repos/pneumonia-chest_xray/test/PNEUMONIA
D:/Repos/pneumonia-chest_xray/val/NORMAL
D:/Repos/pneumonia-chest_xray/val/PNEUMONIA
target
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/pneumonia


In [55]:


source_lista_imagens_com_Covid = []
source_lista_imagens_pneumonia_normal = []
source_lista_imagens_pneumonia_Pneumonia = []

source_lista_imagens_com_Covid = get_imagens_covid(df_covid,dir_source_imagens_Covid)
source_lista_imagens_pneumonia_normal = get_imagens_diretorio(dir_source_pneumonia_normal)
source_lista_imagens_pneumonia_Pneumonia = get_imagens_diretorio(dir_source_pneumonia_pneumonia)

print(len(source_lista_imagens_com_Covid))
print(len(source_lista_imagens_pneumonia_normal))
print(len(source_lista_imagens_pneumonia_Pneumonia))






565
234
390


['D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_477.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_478.jpeg',
 'D:/Repos/pneumonia-chest_xray/test/PNEUMONIA/person100_bacteria_479.jpeg']

## Limpar os diretorios target

In [56]:
clear_diretorio(dir_target_diretorio_imagem_covid)     
clear_diretorio(dir_target_diretorio_pneumonia_pneumonia)       
clear_diretorio(dir_target_diretorio_pneumonia_pneumonia)        
            

In [57]:

def move_imagens_from_source_to_target(source_lista,target):
    for source_path in source_lista:
        imagemName =source_path.split('/')[-1]
        source_target= "{0}/{1}".format(target,imagemName)
    # print(source_path)
    # print(source_target)
        if(os.path.exists(source_path)):
            if not os.path.exists(source_target):
                shutil.copy2(source_path,source_target)
    
    


             
        
 
        

In [60]:
source_lista =source_lista_imagens_com_Covid
target =dir_target_diretorio_imagem_covid

move_imagens_from_source_to_target(source_lista,target)   

source_lista =source_lista_imagens_pneumonia_normal
target =dir_target_diretorio_pneumonia_normal

move_imagens_from_source_to_target(source_lista,target)   


source_lista =source_lista_imagens_pneumonia_Pneumonia
target =dir_target_diretorio_pneumonia_pneumonia

move_imagens_from_source_to_target(source_lista,target)   


## listas auxiliares
